In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re

url_19 = 'https://www.imdb.com/search/title/?count=100&release_date=2019,2019&title_type=feature'
url_18 = 'https://www.imdb.com/search/title/?count=100&release_date=2018,2018&title_type=feature'
url_17 = 'http://www.imdb.com/search/title?count=100&release_date=2017,2017&title_type=feature'

#스크래핑할 데이터: rank, title, description, runtime, genre,
#rating, metascore, votes, gross_earning_in_Mil, director, actor

#스크래핑하여 pandas의 DataFrame으로 변환 후 2017bestfilms.csv 파일로 저장

In [2]:
#open urls
r_19 = requests.get(url_19)
r_18 = requests.get(url_18)
r_17 = requests.get(url_17)

#get BS
soup_19 = BS(r_19.text, 'lxml')
soup_18 = BS(r_18.text, 'lxml')
soup_17 = BS(r_17.text, 'lxml')

In [3]:
#rank
rank_19 = soup_19.select('div > h3 > span.lister-item-index.unbold.text-primary')
rank_19 = [i.text for i in rank_19]

#title
title_19 = soup_19.select('div > h3 > a')
title_19 = [i.text for i in title_19]

In [4]:
#description
#director
#actors
info_19 = soup_19.select('p')
desc_19 = []
direc_actors_19 = []
for i in range(len(info_19)):
    if i % 4 == 1:
        desc_19.append(info_19[i].text.strip())
    if i % 4 == 2:
        direc_actors_19.append(info_19[i].text)

direc_19 = []
actors_19 = []
for i in direc_actors_19:
    tmp = i.split('\n')
    direc_19.append(tmp[2])
    actors_19.append("".join(tmp[-5:-1]))

In [5]:
#runtime
runtime_19 = soup_19.select('div > p > span.runtime')
runtime_19 = [int(i.text.rstrip('min')) for i in runtime_19]

#genre
genre_19 = soup_19.select('div > p > span.genre')
genre_19 = [i.text.strip() for i in genre_19]

In [6]:
'''
#내가 짰던 코드

#rating
#metascore
score_19 = soup_19.select('div > div.lister-item-content > div')
score_19 = [i.text.replace('\n', '').replace(' ', '') for i in score_19]
score_19 = [i.split('Ratethis') for i in score_19]
rating_19 = [i[0] for i in score_19]
score_19 = [i[1].split('X') for i in score_19]
meta_19 = []
for i in range(len(score_19)):
    if len(score_19[i][1]) > 4:
        meta_19.append(score_19[i][1].lstrip()[:2])
    else:
        meta_19.append(np.NaN)
'''

"\n#내가 짰던 코드\n\n#rating\n#metascore\nscore_19 = soup_19.select('div > div.lister-item-content > div')\nscore_19 = [i.text.replace('\n', '').replace(' ', '') for i in score_19]\nscore_19 = [i.split('Ratethis') for i in score_19]\nrating_19 = [i[0] for i in score_19]\nscore_19 = [i[1].split('X') for i in score_19]\nmeta_19 = []\nfor i in range(len(score_19)):\n    if len(score_19[i][1]) > 4:\n        meta_19.append(score_19[i][1].lstrip()[:2])\n    else:\n        meta_19.append(np.NaN)\n"

In [7]:
box_19 = soup_19.select('.lister-item-content')
#각 영화의 요소 다 들어있는 box를 찾기
#그 다음 for문으로 영화 100개에서 요소 하나씩 추출

#rating
rating_19 = []
for i in box_19:
    textline = i.select_one('.inline-block.ratings-imdb-rating').attrs.get("data-value")
    rating_19.append(float(textline))

#metascore
meta_19 = []
for i in box_19:
    textline = i.select_one('.metascore')
    #.metascore 요소 없으면 None 반환
    if textline == None:
        meta_19.append(np.NaN)
        continue
    textline = textline.text
    meta_19.append(int(textline))

In [8]:
#votes
#gross_earning_in_Mil
votes_gross_19 = soup_18.select('div > div.lister-item-content > p.sort-num_votes-visible')
votes_gross_19 = [i.text for i in votes_gross_19]

In [9]:
votes_19 = []
gross_19 = []
for i in range(len(votes_gross_19)):
    tmp = votes_gross_19[i].split('\n')
    votes_19.append(tmp[2])
    if len(tmp) > 4:
        gross_19.append(tmp[-2])
    else:
        gross_19.append(np.NaN)
#metascore나 gross처럼 빈 칸이 있는 경우를 위한 함수 생성도 좋은 방법

In [10]:
df_19 = pd.DataFrame(data = {"rank":rank_19, 'title':title_19, 'description':desc_19, 'runtime(min)':runtime_19, 'genre':genre_19, 'rating':rating_19, 'metascore':meta_19, 'votes':votes_19, 'gross':gross_19, 'director':direc_19, 'actors':actors_19})
df_19 = df_19.set_index('rank')
df_19

,title,description,runtime(min),genre,rating,metascore,votes,gross,director,actors
rank,,,,,,,,,,
1.,Knives Out,A detective investigates the death of a patria...,130,"Comedy, Crime, Drama",7.9,82.0,"58,801",NaN,Rian Johnson,"Daniel Craig, Chris Evans, Ana de Armas, Jamie..."
2.,The Gentlemen,An American expat tries to sell off his highly...,113,"Action, Comedy, Crime",7.9,51.0,"2,761",NaN,Guy Ritchie,"Matthew McConaughey, Charlie Hunnam, Michelle ..."
3.,Once Upon a Time... in Hollywood,A faded television actor and his stunt double ...,161,"Comedy, Drama",7.7,83.0,"276,076",$213.77M,Quentin Tarantino,"Leonardo DiCaprio, Brad Pitt, Margot Robbie, E..."
4.,"Yes, God, Yes","After an innocent AOL chat turns racy, a Catho...",78,"Comedy, Drama",5.9,71.0,"203,102",$49.28M,Karen Maine,"Christian Adam, Susan Blackwell, Alisha Boe, D..."
5.,Joker,"In Gotham City, mentally troubled comedian Art...",122,"Crime, Drama, Thriller",8.5,59.0,"97,834",$15.95M,Todd Phillips,"Joaquin Phoenix, Robert De Niro, Zazie Beetz, ..."
...,...,...,...,...,...,...,...,...,...,...
96.,Official Secrets,The true story of a British whistleblower who ...,112,"Biography, Crime, Drama",7.3,63.0,"106,440",$58.03M,Gavin Hood,"Keira Knightley, Matt Smith, Matthew Goode, Rh..."
97.,Waves,Traces the journey of a suburban family - led ...,135,"Drama, Romance, Sport",7.6,80.0,"62,242",$36.11M,Trey Edward Shults,"Kelvin Harrison Jr., Lucas Hedges, Taylor Russ..."
98.,Long Shot,Journalist Fred Flarsky reunites with his chil...,125,"Comedy, Romance",6.8,67.0,"43,645",$0.72M,Jonathan Levine,"Charlize Theron, Seth Rogen, June Diane Raphae..."


In [11]:
#year 18

rank_18 = soup_18.select('div > h3 > span.lister-item-index.unbold.text-primary')
rank_18 = [i.text for i in rank_18]
title_18 = soup_18.select('div > h3 > a')
title_18 = [i.text for i in title_18]

info_18 = soup_18.select('p')
desc_18 = []
direc_actors_18 = []
for i in range(len(info_18)):
    if i % 4 == 1:
        desc_18.append(info_18[i].text.strip())
    if i % 4 == 2:
        direc_actors_18.append(info_18[i].text)

direc_18 = []
actors_18 = []
for i in direc_actors_18:
    tmp = i.split('\n')
    direc_18.append(tmp[2])
    actors_18.append("".join(tmp[-5:-1]))
    
runtime_18 = soup_18.select('div > p > span.runtime')
runtime_18 = [i.text for i in runtime_18]
genre_18 = soup_18.select('div > p > span.genre')
genre_18 = [i.text.strip() for i in genre_18]

box_18 = soup_18.select('.lister-item-content')
#각 영화의 요소 다 들어있는 box를 찾기
#그 다음 for문으로 영화 100개에서 요소 하나씩 추출

#rating
rating_18 = []
for i in box_18:
    textline = i.select_one('.inline-block.ratings-imdb-rating').attrs.get("data-value")
    rating_18.append(float(textline))

#metascore
meta_18 = []
for i in box_18:
    textline = i.select_one('.metascore')
    #.metascore 요소 없으면 None 반환
    if textline == None:
        meta_18.append(np.NaN)
        continue
    textline = textline.text
    meta_18.append(int(textline))
        
votes_gross_18 = soup_18.select('div > div.lister-item-content > p.sort-num_votes-visible')
votes_gross_18 = [i.text for i in votes_gross_18]

votes_18 = []
gross_18 = []
for i in range(len(votes_gross_18)):
    tmp = votes_gross_18[i].split('\n')
    votes_18.append(tmp[2])
    if len(tmp) > 4:
        gross_18.append(tmp[-2])
    else:
        gross_18.append(np.NaN)

df_18 = pd.DataFrame(data = {"rank":rank_18, 'title':title_18, 'description':desc_18, 'runtime':runtime_18, 'genre':genre_18, 'rating':rating_18, 'metascore':meta_18, 'votes':votes_18, 'gross':gross_18, 'director':direc_18, 'actors':actors_18})
df_18 = df_18.set_index('rank')
df_18

,title,description,runtime,genre,rating,metascore,votes,gross,director,actors
rank,,,,,,,,,,
1.,The Kissing Booth,A high school student is forced to confront he...,105 min,"Comedy, Romance",6.0,NaN,"58,801",NaN,Vince Marcello,"Joey King, Jacob Elordi, Joel Courtney, Megan ..."
2.,MILF,Three best friends in their forties start a re...,101 min,Comedy,4.7,NaN,"2,761",NaN,Axelle Laffont,"Marie-Josée Croze, Virginie Ledoyen, Axelle La..."
3.,Solo: A Star Wars Story,During an adventure into the criminal underwor...,135 min,"Action, Adventure, Sci-Fi",6.9,62.0,"276,076",$213.77M,Ron Howard,"Alden Ehrenreich, Woody Harrelson, Emilia Clar..."
4.,BlacKkKlansman,"Ron Stallworth, an African American police off...",135 min,"Biography, Crime, Drama",7.5,83.0,"203,102",$49.28M,Spike Lee,"John David Washington, Adam Driver, Laura Harr..."
5.,Mortal Engines,In a post-apocalyptic world where cities ride ...,128 min,"Action, Adventure, Fantasy",6.1,44.0,"97,834",$15.95M,Christian Rivers,"Hera Hilmar, Robert Sheehan, Hugo Weaving, Jihae"
...,...,...,...,...,...,...,...,...,...,...
96.,Maze Runner: The Death Cure,Young hero Thomas embarks on a mission to find...,143 min,"Action, Adventure, Sci-Fi",6.2,50.0,"106,440",$58.03M,Wes Ball,"Dylan O'Brien, Ki Hong Lee, Kaya Scodelario, T..."
97.,Mile 22,A small team of elite American intelligence of...,94 min,"Action, Thriller",6.1,38.0,"62,242",$36.11M,Peter Berg,"Mark Wahlberg, Lauren Cohan, Iko Uwais, John M..."
98.,Beoning,Jong-su bumps into a girl who used to live in ...,148 min,"Drama, Mystery",7.5,90.0,"43,645",$0.72M,Chang-dong Lee,"Ah-In Yoo, Steven Yeun, Jong-seo Jun, Soo-Kyun..."


In [15]:
#year 17

rank_17 = soup_17.select('div > h3 > span.lister-item-index.unbold.text-primary')
rank_17 = [i.text for i in rank_17]
title_17 = soup_17.select('div > h3 > a')
title_17 = [i.text for i in title_17]

info_17 = soup_17.select('p')
desc_17 = []
direc_actors_17 = []
for i in range(len(info_17)):
    if i % 4 == 1:
        desc_17.append(info_17[i].text.strip())
    if i % 4 == 2:
        direc_actors_17.append(info_17[i].text)

direc_17 = []
actors_17 = []
for i in direc_actors_17:
    tmp = i.split('\n')
    direc_17.append(tmp[2])
    actors_17.append("".join(tmp[-5:-1]))
    
runtime_17 = soup_17.select('div > p > span.runtime')
runtime_17 = [i.text for i in runtime_17]
genre_17 = soup_17.select('div > p > span.genre')
genre_17 = [i.text.strip() for i in genre_17]

box_17 = soup_17.select('.lister-item-content')
#각 영화의 요소 다 들어있는 box를 찾기
#그 다음 for문으로 영화 100개에서 요소 하나씩 추출

#rating
rating_17 = []
for i in box_17:
    textline = i.select_one('.inline-block.ratings-imdb-rating').attrs.get("data-value")
    rating_17.append(float(textline))

#metascore
meta_17 = []
for i in box_17:
    textline = i.select_one('.metascore')
    #.metascore 요소 없으면 None 반환
    if textline == None:
        meta_17.append(np.NaN)
        continue
    textline = textline.text
    meta_17.append(int(textline))
        
votes_gross_17 = soup_17.select('div > div.lister-item-content > p.sort-num_votes-visible')
votes_gross_17 = [i.text for i in votes_gross_17]

votes_17 = []
gross_17 = []
for i in range(len(votes_gross_17)):
    tmp = votes_gross_17[i].split('\n')
    votes_17.append(tmp[2])
    if len(tmp) > 4:
        gross_17.append(tmp[-2])
    else:
        gross_17.append(np.NaN)

df_17 = pd.DataFrame(data = {"rank":rank_17, 'title':title_17, 'description':desc_17, 'runtime':runtime_17, 'genre':genre_17, 'rating':rating_17, 'metascore':meta_17, 'votes':votes_17, 'gross':gross_17, 'director':direc_17, 'actors':actors_17})
df_17 = df_17.set_index('rank')
df_17

,title,description,runtime,genre,rating,metascore,votes,gross,director,actors
rank,,,,,,,,,,
1.,Animal Crackers,A family must use a magical box of Animal Crac...,105 min,"Animation, Adventure, Comedy",5.8,60.0,"1,373",NaN,"Tony Bancroft,","Emily Blunt, Danny DeVito, John Krasinski, Ian..."
2.,"Three Billboards Outside Ebbing, Missouri",A mother personally challenges the local autho...,115 min,"Comedy, Crime, Drama",8.2,88.0,"408,581",$54.51M,Martin McDonagh,"Frances McDormand, Woody Harrelson, Sam Rockwe..."
3.,Blade Runner 2049,Young Blade Runner K's discovery of a long-bur...,164 min,"Action, Drama, Mystery",8.0,81.0,"437,891",$92.05M,Denis Villeneuve,"Harrison Ford, Ryan Gosling, Ana de Armas, Dav..."
4.,The Post,A cover-up spanning four U.S. Presidents pushe...,116 min,"Drama, Thriller",7.2,83.0,"129,153",$81.90M,Steven Spielberg,"Meryl Streep, Tom Hanks, Sarah Paulson, Bob Od..."
5.,Pirates of the Caribbean: Dead Men Tell No Tales,Captain Jack Sparrow is pursued by old rival C...,129 min,"Action, Adventure, Fantasy",6.6,39.0,"253,240",$172.56M,"Joachim Rønning,","Johnny Depp, Geoffrey Rush, Javier Bardem, Orl..."
...,...,...,...,...,...,...,...,...,...,...
96.,47 Meters Down,Two sisters vacationing in Mexico are trapped ...,89 min,"Adventure, Drama, Horror",5.6,52.0,"47,253",$44.31M,Johannes Roberts,"Mandy Moore, Claire Holt, Matthew Modine, Chri..."
97.,American Made,"The story of Barry Seal, an American pilot who...",115 min,"Action, Biography, Comedy",7.2,65.0,"152,581",$51.34M,Doug Liman,"Tom Cruise, Domhnall Gleeson, Sarah Wright, Je..."
98.,Death Note,A high school student named Light Turner disco...,101 min,"Action, Adventure, Crime",4.5,43.0,"73,317",NaN,Adam Wingard,"Nat Wolff, LaKeith Stanfield, Margaret Qualley..."


In [13]:
df_17.to_csv('C:\\Users\\student\\mypy\\data\\output\\2017bestfilms.csv')
df_18.to_csv('C:\\Users\\student\\mypy\\data\\output\\2018bestfilms.csv')
df_19.to_csv('C:\\Users\\student\\mypy\\data\\output\\2019bestfilms.csv')